In [4]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import threading
import psycopg2
import psycopg2.extensions

def connect():
    conn = psycopg2.connect(
        database="postgres", 
        user="student", 
        password="HSUnivPGTests981", 
            host="database-1.c10eiwo8yylk.ap-south-1.rds.amazonaws.com", 
        port="5432")
    return conn


def setup_database():
    conn = connect()
    cur = conn.cursor()
    # Create table
    cur.execute('DROP TABLE IF EXISTS non_repeatable_read_demo;')
    cur.execute('CREATE TABLE non_repeatable_read_demo (id SERIAL PRIMARY KEY, value VARCHAR(100));')
    cur.execute("INSERT INTO non_repeatable_read_demo (value) VALUES ('Initial value');")
    conn.commit()
    conn.close()
    
# Transaction 1 - This transaction will read the same record twice, with a delay
def transaction1(isolation_level):
    conn = connect()
    conn.set_isolation_level(isolation_level)
    cur = conn.cursor()
    # First read
    cur.execute('SELECT * FROM non_repeatable_read_demo WHERE id = 1;')
    row1 = cur.fetchone()
    print('Transaction 1 - First read:', row1)
    
    # Simulate some delay
    import time
    time.sleep(5)
    
    # Second read
    cur.execute('SELECT * FROM non_repeatable_read_demo WHERE id = 1;')
    row2 = cur.fetchone()
    print('Transaction 1 - Second read:', row2)
    
    if row1[1]!=row2[1]:
        print("Transaction 1 - Non repeatable read anomaly detected! '{}' changed to '{}'".format(row1[1],row2[1]))
    else:
        print("Transaction 1 - No non repeatable read anomaly. '{}' equal to '{}'".format(row1[1],row2[1]))
        
    
    conn.close()   
    
# Transaction 2 - This transaction will update the record that Transaction 1 is reading
def transaction2(isolation_level):
    conn = connect()
    conn.set_isolation_level(isolation_level)
    cur = conn.cursor()
    # Simulate a delay to ensure Transaction 1 reads the row before the update
    import time
    time.sleep(2)
    
    # Update the row
    cur.execute("UPDATE non_repeatable_read_demo SET value = 'Updated value' WHERE id = 1;")
    conn.commit()
    print('Transaction 2 - Updated the row')
    conn.close()

setup_database()

# isolation_level = psycopg2.extensions.ISOLATION_LEVEL_READ_COMMITTED
isolation_level = psycopg2.extensions.ISOLATION_LEVEL_REPEATABLE_READ
#isolation_level = psycopg2.extensions.ISOLATION_LEVEL_SERIALIZABLE
print(isolation_level)

# Running transactions in parallel threads
thread1 = threading.Thread(target=transaction1, args=(isolation_level,))
thread2 = threading.Thread(target=transaction2, args=(isolation_level,))

thread1.start()
thread2.start()

thread1.join()
thread2.join()



2
Transaction 1 - First read: (1, 'Initial value')
Transaction 2 - Updated the row
Transaction 1 - Second read: (1, 'Initial value')
Transaction 1 - No non repeatable read anomaly. 'Initial value' equal to 'Initial value'


In [10]:
import threading
import psycopg2
import psycopg2.extensions

def setup_database():
    conn = connect()
    cur = conn.cursor()
    # Create table
    cur.execute('DROP TABLE IF EXISTS lost_update_demo;')
    cur.execute('CREATE TABLE lost_update_demo (id SERIAL PRIMARY KEY, value int);')
    cur.execute("INSERT INTO lost_update_demo (value) VALUES (0);")
    conn.commit()
    conn.close()
    
# Transaction 1 - This transaction will read the same record twice, with a delay
def transaction1(isolation_level, num):
    conn = connect()
    conn.set_isolation_level(isolation_level)
    cur = conn.cursor()
    # First read
    cur.execute('select value from lost_update_demo limit 1;')
    row1 = cur.fetchone()
    n=row1[0] 

    print("Transaction {} - Before update: {},{}".format(num,row1,n))
    
    # Simulate some delay
    import time
    time.sleep(num)
    
    cur.execute('UPDATE lost_update_demo SET value = value+1;')
    
    # Simulate some delay
    import time
    time.sleep(2)
    
    cur.execute('select value from lost_update_demo limit 1;')
    row1 = cur.fetchone()
    print("Transaction {} - After update: {}".format(num,row1))
    
    conn.commit()
    conn.close()         

setup_database()

# isolation_level = psycopg2.extensions.ISOLATION_LEVEL_READ_COMMITTED
# isolation_level = psycopg2.extensions.ISOLATION_LEVEL_REPEATABLE_READ
isolation_level = psycopg2.extensions.ISOLATION_LEVEL_SERIALIZABLE

print('Isolation level is ', isolation_level)

# Running transactions in parallel threads
thread2 = threading.Thread(target=transaction1, args=(isolation_level,2))
thread1 = threading.Thread(target=transaction1, args=(isolation_level,1))

thread1.start()
thread2.start()

thread1.join()
thread2.join()



Isolation level is  3
Transaction 2 - Before update: (0,),0
Transaction 1 - Before update: (0,),0


Exception in thread Exception in thread Thread-13 (transaction1):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
Thread-14 (transaction1):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/Users/levandalbashvili/Documents/GitHub/databaseclass/myenv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    self.run()
  File "/Users/levandalbashvili/Documents/GitHub/databaseclass/myenv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1012, in run
    _threading_Thread_run(self)
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/q3/y9cmlg4s4xs988k0ck20g8cc0000gn/T/ipykernel_99346/2537410951.py", line 

In [13]:
import threading
import psycopg2
import psycopg2.extensions
import time

def setup_database():
    conn = connect()
    cur = conn.cursor()
    # Create table
    cur.execute('DROP TABLE IF EXISTS serialization_anomaly_demo;')
    cur.execute('CREATE TABLE serialization_anomaly_demo (id SERIAL PRIMARY KEY, value int);')
    cur.execute("INSERT INTO serialization_anomaly_demo (value) VALUES (1);")
    conn.commit()
    conn.close()
    
# Transaction 1 - This transaction will read the same record twice, with a delay
def transaction1(isolation_level, num):
    conn = connect()
    conn.set_isolation_level(isolation_level)
    cur = conn.cursor()
    #check
    cur.execute('select sum(value) from serialization_anomaly_demo;')
    row1 = cur.fetchone()
    print("Transaction {}, check: {}".format(num,row1))
    sumv = row1[0]

    # Simulate some delay
    
    time.sleep(5)
    # First read
    if sumv<2:
        cur.execute("INSERT INTO serialization_anomaly_demo (value) VALUES (1);")
    cur.execute('select sum(value) from serialization_anomaly_demo;')
    row2 = cur.fetchone()
    print("Transaction {} after insert: {}".format(num,row2))
      
    conn.commit()
    conn.close() 
        

setup_database()

# isolation_level = psycopg2.extensions.ISOLATION_LEVEL_READ_COMMITTED
# isolation_level = psycopg2.extensions.ISOLATION_LEVEL_REPEATABLE_READ
isolation_level = psycopg2.extensions.ISOLATION_LEVEL_SERIALIZABLE

print('Isolation level is ', isolation_level)

# Running transactions in parallel threads
thread2 = threading.Thread(target=transaction1, args=(isolation_level,2))

thread1 = threading.Thread(target=transaction1, args=(isolation_level,1))

thread1.start()
time.sleep(1)
thread2.start()

thread1.join()
thread2.join()




Isolation level is  3
Transaction 1, check: (1,)
Transaction 2, check: (1,)
Transaction 1 after insert: (2,)


Exception in thread Thread-19 (transaction1):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/Users/levandalbashvili/Documents/GitHub/databaseclass/myenv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/anaconda3/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/q3/y9cmlg4s4xs988k0ck20g8cc0000gn/T/ipykernel_99346/4001602051.py", line 32, in transaction1
psycopg2.errors.SerializationFailure: could not serialize access due to read/write dependencies among transactions
DETAIL:  Reason code: Canceled on identification as a pivot, during write.
HINT:  The transaction might succeed if retried.

